In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [63]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
training_data = datasets.FashionMNIST(root='data', download=True, train=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='data', download=True, train=False, transform=transforms.ToTensor())

In [5]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [6]:
len(train_dataloader), len(test_dataloader), 938*64, 157*64

(938, 157, 60032, 10048)

In [7]:
len(train_dataloader.dataset), len(test_dataloader.dataset), len(test_dataloader.dataset) // 64, len(test_dataloader)

(60000, 10000, 156, 157)

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=64, bias=True)
    (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [9]:
learning_rate, epochs, batch_size = 1e-3, 5, 64

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss : {loss:>7f}  [{current:>5d} / {size:>5d}]")

In [12]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)
            test_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error :\nAccuracy:{(100 * correct):>0.1f}%, Avg loss : {test_loss:>8f}\n")

In [13]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------------------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------------------------------------------
loss : 2.287653  [    0 / 60000]
loss : 0.536510  [ 6400 / 60000]
loss : 0.396541  [12800 / 60000]
loss : 0.500711  [19200 / 60000]
loss : 0.458946  [25600 / 60000]
loss : 0.399245  [32000 / 60000]
loss : 0.343323  [38400 / 60000]
loss : 0.472996  [44800 / 60000]
loss : 0.418914  [51200 / 60000]
loss : 0.420621  [57600 / 60000]
Test Error :
Accuracy:84.7%, Avg loss : 0.412872

Epoch 2
-----------------------------------------------------------
loss : 0.359216  [    0 / 60000]
loss : 0.366931  [ 6400 / 60000]
loss : 0.278142  [12800 / 60000]
loss : 0.309142  [19200 / 60000]
loss : 0.348983  [25600 / 60000]
loss : 0.294572  [32000 / 60000]
loss : 0.264177  [38400 / 60000]
loss : 0.367755  [44800 / 60000]
loss : 0.343789  [51200 / 60000]
loss : 0.363483  [57600 / 60000]
Test Error :
Accuracy:85.8%, Avg loss : 0.387499

Epoch 3
-----------------------------------------------------------
loss : 0.264387  [    0 / 60000]
los

In [37]:
import numpy as np
import matplotlib.pyplot as plt

In [38]:
labels = np.arange(10)
tags = ['T-Shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']
label_tags = dict(zip(labels, tags))
print(label_tags)

{0: 'T-Shirt', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle Boot'}


In [39]:
columns, rows = 6, 6
fig = plt.figure(figsize=(15, 15))

<Figure size 1500x1500 with 0 Axes>

In [66]:
for i in range(1, columns*rows+1):
    data_idx = np.random.randint(len(test_dataloader))
    input_img = test_data[data_idx][0].to(device)
    #input_img = test_data[data_idx][0]unsqueeze(dim=0)
    
    output = model.eval(input_img)
    _, argmax = torch.max(output, 1)
    pred = label_tags[argmax.item()]
    label = label_tags[test_data[data_idx][1]]
    
    fig.add_subplot(rows, columns, i)
    if pred == label:
        plt.title(pred + ' , right!')
        cmap = 'Blue'
    else:
        plt.title('Not ' + pred + ' but ' + label)
        cmap = 'Reds'
        
    plot_img = test_data[data_idx][0][0,:,:]
    plt.imshow(plot_img, cmap=cmap)
    plt.axis('off')
plt.show()

TypeError: eval() takes 1 positional argument but 2 were given

In [57]:
test_data[3][0].to(device).unsqueeze(dim=0).shape

torch.Size([1, 1, 28, 28])

In [36]:
torch.manual_seed(2023)

a = torch.randn(5, 3)
print(a.shape)
print()
print(a)
print()

val, idx = torch.max(a, 0)
print(val)
print(idx)
print()

val, idx = torch.min(a, 1)
print(val)
print(idx)

torch.Size([5, 3])

tensor([[-1.2075,  0.5493, -0.3856],
        [ 0.6910, -0.7424,  0.1570],
        [ 0.0721,  1.1055,  0.2218],
        [-0.0794, -1.0846, -1.5421],
        [ 0.9377, -0.9787,  2.0930]])

tensor([0.9377, 1.1055, 2.0930])
tensor([4, 2, 4])

tensor([-1.2075, -0.7424,  0.0721, -1.5421, -0.9787])
tensor([0, 1, 0, 2, 1])
